In [18]:
from src.inference.segment_inference import inference
import cv2
from pytesseract import pytesseract

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

pytesseract.tesseract_cmd = path_to_tesseract

In [19]:
def crop_image(image, box):

    x1, y1, x2, y2 = map(int, box)  # Convert coordinates to integers
    cropped_image = image[y1:y2, x1:x2]
    return cropped_image


def extract_text(outputs, image_origin):
    for i in range(len(outputs)):
        image = crop_image(image_origin, outputs[i].get("box"))
        text = pytesseract.image_to_string(image)

        outputs[i].update({"image": image})
        outputs[i].update({"text": text})
        if "text" in outputs[i]:
            outputs[i]["text"] += text
        else:
            outputs[i].update({"text": text})
    return outputs

In [20]:
image_origin = cv2.imread("./src/data/resume-for-fresher-template-281.jpg")
outputs = inference(
    image_origin,
    model_path="./src/model/segment.onnx",
    threshold_confidence=0.6,
    threshold_iou=0.7,
)
outputs = extract_text(outputs,image_origin=image_origin)

In [21]:
for i in range(len(outputs)):
    cv2.imshow((outputs[i].get("label").upper()),outputs[i].get("image"))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

In [22]:
def extract_text_dict(outputs):
    result_dict = {}
    for output in outputs:
        label = output.get("label").lower()
        text = output.get("text")
        if label in result_dict:
            result_dict[label] += " " + text
        else:
            result_dict[label] = text
    return result_dict
outputs = extract_text_dict(outputs)
outputs

{'experience': 'PROFESSIONAL EXPERIENCE\n\nSoftware Developer Intern\nXYZ Tech Solutions — City, State | 20Xx - 20xx\n\n* Collaborated with a cross-functional team to\ndevelop software solutions for client projects.\n\n‘+ Assisted in troubleshooting and debugging code,\ncontributing to timely delivery.\n\n+ Patticipated in code reviews.\n\nIT Support Intern\n‘ABC Company ~ City, State | 20XX - 20xX\n\n+ Provided technical support to employees,\nresolving hardware and software issue.\n\n‘+ Assisted in the maintenance and troubleshooting\nof the company’s IT infrastructure.\n\n+ Conducted user training sessions\n\nMobile App Development Project\nUniversity of Tech — City, State | 20XX - 20xX\n\n‘+ Spearheaded the development of a mobile\napplication aimed at enhancing campus\n‘communication,\n\n+ Utilized Java and Android Studio to design and\nimplement user-friendly interfaces.\nPROFESSIONAL EXPERIENCE\n\nSoftware Developer Intern\nXYZ Tech Solutions — City, State | 20Xx - 20xx\n\n* Col

In [23]:
print(outputs.get('EXPERIENCE'))

None
